In [ ]:
import thermomatch as match
match.ThermoImpexGenerator.create_local_database_if_not_existent = True

In [ ]:
# Set path to `schemas` and `lua` not from the configuration file
match.ThermoImpexGenerator.setResourcesDirectory("../../Resources")
# Connect to Arango database
generator = match.ThermoImpexGenerator("http://localhost:8529", "root", "", "ORD_thermochimie12a", True, match.ThermoImpexGenerator.File, True)

In [ ]:
# Import References (the collection name and VertexDataSource schema would be obtained from the script by default)
keys = generator.importFileToDatabase("FormatStructDataFile", "scripts-in/references.FormatStructDataFile.json", "data-in/ref_thermochimie.json", "")  #- bug

In [ ]:
# List of implemented thermodynamic formats
#print( generator.allFormats() )
generator = match.ThermoImpexGenerator("http://localhost:8529", "root", "", "ORD_thermochimie12a", True, match.ThermoImpexGenerator.File, True) # bug in PhreeqcImportData.cpp line 510 when loading datasourceVertex db - therefore database needs to be reloaded by uncommenting this line

# Execute import from Phreeqc format data
description = [
    match.ImpexRowDescription("SOLUTION_MASTER_SPECIES", "VertexElement", "records", "FormatTableFile", "scripts-in/elements.FormatTableFile.json"), # element records
    match.ImpexRowDescription("SOLUTION_SPECIES", "VertexSubstance", "records", "FormatKeyValueFile", "scripts-in/master_aqsubstances.FormatKeyValueFile.json"), # master species substance records
    match.ImpexRowDescription("SECONDARY_MASTER_SPECIES", "VertexSubstance", "records", "FormatKeyValueFile", "scripts-in/product_aqsubstances.FormatKeyValueFile.json"), # aqueous species substance records
    match.ImpexRowDescription("SECONDARY_MASTER_SPECIES", "VertexReaction", "records_and_links", "FormatKeyValueFile", "scripts-in/reaction_aqsubstances.FormatKeyValueFile.json"), # aqueous species reaction records and lniks to reactants
    match.ImpexRowDescription("PHASES", "VertexSubstance", "records", "FormatKeyValueFile", "scripts-in/product_solidsubstances.FormatKeyValueFile.json"), # solid substance records
    match.ImpexRowDescription("PHASES", "VertexReaction", "records_and_links", "FormatKeyValueFile", "scripts-in/reaction_solidsubstances.FormatKeyValueFile.json"), # solid reaction records and links to reactants
    match.ImpexRowDescription("PHASES", "VertexPhase", "records_and_links", "FormatKeyValueFile", "scripts-in/phase_solidsubstances.FormatKeyValueFile.json"), # solid phase records and link to substances (endmembers)
    match.ImpexRowDescription("SIT", "VertexPhase", "records", "FormatTableFile", "scripts-in/phase_sit.FormatTableFile.json"), # aq_sit phase record
    match.ImpexRowDescription("SIT", "VertexMixingModel", "records", "FormatTableFile", "scripts-in/mixmod_sit.FormatTableFile.json"), # sit mixmod record
    match.ImpexRowDescription("SIT", "VertexInteraction", "records_and_links", "FormatTableFile", "scripts-in/interaction_sit.FormatTableFile.json"), # interaction records and link to species
    match.ImpexRowDescription("refs", "VertexDataSource", "records_and_links", "FormatStructDataFile", "scripts-in/references.FormatStructDataFile.json"), # makes links between references and reaction records
    match.ImpexRowDescription("ThermoDataSet", "VertexThermoDataSet", "records_and_links", "FormatTableFile", "scripts-in/thermodataset.FormatTableFile.json") # ThermoDataSet record and link to elements, substances, rdc
]

### Preparing file for import
The original PhreeqC_SIT_electron_10a_22Nov2018.dat that can be downloaded from the thermochimie website does not contain `SECONDARY_MASTER_SPECIES` key. For the script to work add:

`SECONDARY_MASTER_SPECIES` after
```
1.000Zr+4     = Zr+4
     log_k     0.000     
     delta_h   0.000     #kJ/mol        
     # Enthalpy of formation:           -608.5        #kJ/mol        #05BRO/CUR
     -analytic 0E+0 0E+0 0E+0 0E+0 0E+0

In [ ]:
#print( description ) ~1 min
#generator.runImport("PhreeqcImport", description, "data-in/PhreeqC_SIT_electron_10a_22Nov2018.dat", "rdc_THERMOCHIMIE10a");
generator.runImport("PhreeqcImport", description, "data-in/PHREEQC_SIT_e-_ThermoChimie_v12a.dat", "rdc_THERMOCHIMIE12a");

In [ ]:
# update data of elements from CODATA, NEA and SUCPRT
fields_list = [ "_id", "properties.symbol" ]
generator.updateFileToDatabase("FormatTableFile", "scripts-in/elements_update.FormatTableFile.json",
                      "../../datatypes/elements/data-in/elements.csv", "", "", fields_list )